In [3]:
import pandas as pd
import json
import time
from dotenv import load_dotenv
import os

load_dotenv()  # This loads the variables from .env into the environment

True

In [4]:
from openai import OpenAI


client = OpenAI()
#from error import APIConnectionError

In [5]:
df=pd.read_csv('../App/data/Mental_Health_FAQ.csv')

In [6]:
df.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [7]:
documents = df.to_dict(orient='records')
documents[0:2]

[{'Question_ID': 1590140,
  'Questions': 'What does it mean to have a mental illness?',
  'Answers': 'Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical con

In [14]:
prompt_template = """
You emulate a user of our mental health psychiatrist application.
Formulate 5 questions this user might ask based on the answers in this database.
Make the questions specific to that chunk.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

ANSWER: {Answers}


Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [15]:
documents[0]

{'Question_ID': 1590140,
 'Questions': 'What does it mean to have a mental illness?',
 'Answers': 'Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical condit

In [16]:
prompt = prompt_template.format(**documents[0])

In [17]:
prompt

'You emulate a user of our mental health psychiatrist application.\nFormulate 5 questions this user might ask based on the answers in this database.\nMake the questions specific to that chunk.\nThe record should contain the answer to the questions, and the questions should\nbe complete and not too short. Use as fewer words as possible from the record. \n\nThe record:\n\nANSWER: Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may requ

In [18]:
print(_)

You emulate a user of our mental health psychiatrist application.
Formulate 5 questions this user might ask based on the answers in this database.
Make the questions specific to that chunk.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

ANSWER: Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.
Mental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that t

In [19]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [20]:
questions = llm(prompt)
questions

'{\n    "questions": [\n        "What defines mental illnesses according to your understanding?",\n        "Can mild mental illnesses only affect certain aspects of life or can they be more pervasive?",\n        "Which serious mental illnesses are mentioned that may require hospitalization for care?",\n        "How are mental illnesses compared to disorders like diabetes in terms of their medical nature?",\n        "What is the effectiveness rate of treatments for mental illnesses and how do they impact life quality?"\n    ]\n}'

In [21]:
import json

In [22]:
json.loads(questions)

{'questions': ['What defines mental illnesses according to your understanding?',
  'Can mild mental illnesses only affect certain aspects of life or can they be more pervasive?',
  'Which serious mental illnesses are mentioned that may require hospitalization for care?',
  'How are mental illnesses compared to disorders like diabetes in terms of their medical nature?',
  'What is the effectiveness rate of treatments for mental illnesses and how do they impact life quality?']}

In [27]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [28]:
from tqdm.auto import tqdm

In [25]:
results = {}


In [29]:
len(documents)
documents[0]

{'Question_ID': 1590140,
 'Questions': 'What does it mean to have a mental illness?',
 'Answers': 'Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. Just as diabetes is a disorder of the pancreas, mental illness is a medical condit

## Parallelarization


In [30]:
from concurrent.futures import ThreadPoolExecutor, as_completed
def process_document(doc):
    doc_id = doc['Question_ID']
    if doc_id in results:
        return None  # Skip this document if already processed
    try:
        questions_raw = generate_questions(doc)
        questions = json.loads(questions_raw)
        return (doc_id, questions['questions'])
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing doc_id {doc_id}: {e}")
        return None

# Track time
t0 = time.time()

# Create a ThreadPoolExecutor with a suitable number of workers
with ThreadPoolExecutor(max_workers=3) as executor:
    # Submit all tasks
    
    futures = {executor.submit(process_document, doc): doc for doc in tqdm(documents)}

    total_tasks = len(futures)
    # Collect results as they complete
    for future in tqdm(as_completed(futures)):
        result = future.result()
        if result is not None:
            doc_id, questions = result
            results[doc_id] = questions

# End time
t1 = time.time()
# Print time taken
print(f"Time taken: {(t1 - t0)/60} minutes")


  0%|          | 0/98 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Error processing doc_id 6521784: Expecting ',' delimiter: line 1 column 341 (char 340)
Time taken: 0.8633713722229004 minutes


In [32]:
len(results)


97

In [55]:
# no need to use the following code

In [56]:
# import time
# t0=time.time()
# for doc in tqdm(random_docs[:1000]): 
#     doc_id = doc['id']
#     if doc_id in results:
#         continue
#     try:
#         questions_raw = generate_questions(doc)
#         questions = json.loads(questions_raw)  # If an error happens here, the next line will not run
#         results[doc_id] = questions['questions']  # This line will be skipped if there's an error in json.loads()
#     except (json.JSONDecodeError, KeyError) as e:
#         print(f"Error processing doc_id {doc_id}: {e}")
    
# t1=time.time()
# print((t0-t1)/60)

In [37]:
len(results)

97

In [34]:
final_results = []

for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

In [35]:
final_results[0:10]

[(1590140, 'What are the different levels of severity for mental illnesses?'),
 (1590140, 'Can you explain how mental illnesses impact daily functioning?'),
 (1590140,
  "Are mental illnesses linked to a person's character or intelligence?"),
 (1590140, 'What treatments are available for mental illnesses?'),
 (1590140, 'How effective is treatment for individuals with mental illness?'),
 (6361820, 'What percentage of mental health conditions develop by age 24?'),
 (6361820, 'Which demographic is most susceptible to eating disorders?'),
 (6361820,
  'How are serious emotional disturbances defined in children under 18?'),
 (6361820,
  'Why is it difficult to identify mental health disorders in adolescents?'),
 (6361820,
  'What is the estimated proportion of adults affected by mental illness in America?')]

In [38]:
len(final_results)

485

In [39]:
df = pd.DataFrame(final_results, columns=['id', 'question'])
df.head()

,id,question
0,1590140,What are the different levels of severity for ...
1,1590140,Can you explain how mental illnesses impact da...
2,1590140,Are mental illnesses linked to a person's char...
3,1590140,What treatments are available for mental illne...
4,1590140,How effective is treatment for individuals wit...


In [40]:
df.to_csv('../App/data/ground-truth-data.csv', index=False)

In [42]:
!head '../App/data/ground-truth-data.csv'

id,question
1590140,What are the different levels of severity for mental illnesses?
1590140,Can you explain how mental illnesses impact daily functioning?
1590140,Are mental illnesses linked to a person's character or intelligence?
1590140,What treatments are available for mental illnesses?
1590140,How effective is treatment for individuals with mental illness?
6361820,What percentage of mental health conditions develop by age 24?
6361820,Which demographic is most susceptible to eating disorders?
6361820,How are serious emotional disturbances defined in children under 18?
6361820,Why is it difficult to identify mental health disorders in adolescents?
